In [ ]:
# Q. 타이타닉 생존자 예측 데이터 세트 train.csv에 대하여 다음 사항을 수행하세요.
# - 일괄 전처리 사용자 함수 transform_features(df) 작성

# - 1. dt, lr, rf 모델링 및 평가(정확도)

# - 2. dt_clf , folds=5 적용하여 KFold 교차검증 수행

# - 3. dt_clf , cv=5 적용, cross_val_score를 이용하여 교차검증 수행

# - 4. GridSearchCV의 최적 하이퍼 파라미터로 학습된 Estimator로 예측 및 평가 수행.
#   - parameters = {'max_depth':[2,3,5,10], 'min_samples_split':[2,3,5], 'min_samples_leaf':[1,5,8]}
#   - dt_clf, scoring='accuracy', cv=5 적용



import pandas as pd

df = pd.read_csv('train.csv')

print(df.head(5))


   PassengerId  Survived  Pclass  \
0            1         0       3   
1            2         1       1   
2            3         1       3   
3            4         1       1   
4            5         0       3   

                                                Name     Sex   Age  SibSp  \
0                            Braund, Mr. Owen Harris    male  22.0      1   
1  Cumings, Mrs. John Bradley (Florence Briggs Th...  female  38.0      1   
2                             Heikkinen, Miss. Laina  female  26.0      0   
3       Futrelle, Mrs. Jacques Heath (Lily May Peel)  female  35.0      1   
4                           Allen, Mr. William Henry    male  35.0      0   

   Parch            Ticket     Fare Cabin Embarked  
0      0         A/5 21171   7.2500   NaN        S  
1      0          PC 17599  71.2833   C85        C  
2      0  STON/O2. 3101282   7.9250   NaN        S  
3      0            113803  53.1000  C123        S  
4      0            373450   8.0500   NaN        S  


In [1]:
# - 1. dt, lr, rf 모델링 및 평가(정확도)

import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score
import warnings
warnings.filterwarnings('ignore')

#결측치 Null 처리, age는 평균, cabin 은 null, embarked null, fare 0

def fillna(df):
    df['Age'].fillna(df['Age'].mean(), inplace=True)
    df['Cabin'].fillna('N', inplace=True)
    df['Embarked'].fillna('N', inplace=True)
    df['Fare'].fillna(0, inplace=True)
    return df

#머신러닝 알고리즘에 불필요한 속성 제거, passengerid, name, ticket
def drop_features(df):
    df.drop(['PassengerId', 'Name', 'Ticket'], axis = 1, inplace=True)
    return df

#레이블 인코딩 수행 cabin 선실번호 첫문자만 추출 후 인코딩, sex, embarked 사용
def format_features(df):
    df['Cabin'] = df['Cabin'].str[:1]
    features = ['Cabin', 'Sex', 'Embarked']
    for feature in features:
        le = LabelEncoder()
        le = le.fit(df[feature])
        df[feature] = le.transform(df[feature])
    
    return df

# 사용자 함수 한번에 실행
def transform_features(df):
    df = fillna(df)
    df = drop_features(df)
    df = format_features(df)
    return df

# 타이타닉 데이터셋 로드 및 전처리
titanic_df = pd.read_csv('train.csv')
X_titanic_df = titanic_df.drop('Survived', axis=1)
y_titanic_df = titanic_df['Survived']

# 위에서 설정한 사용자 함수 실행
X_titanic_df = transform_features(X_titanic_df)

X_train, X_test, y_train, y_test = train_test_split(X_titanic_df, y_titanic_df, test_size=0.2, random_state=11)

# 모델 생성 및 평가 함수
def evaluate_model(model, X_train, X_test, y_train, y_test):
    model.fit(X_train, y_train)
    pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, pred)
    precision = precision_score(y_test, pred)
    recall = recall_score(y_test, pred)
    return accuracy, precision, recall

# 결정 트리, 로지스틱 회귀, 랜덤 포레스트 모델 생성 및 평가
dt_clf = DecisionTreeClassifier(random_state=11)
lr_clf = LogisticRegression()
rf_clf = RandomForestClassifier(random_state=11)

dt_accuracy, dt_precision, dt_recall = evaluate_model(dt_clf, X_train, X_test, y_train, y_test)
lr_accuracy, lr_precision, lr_recall = evaluate_model(lr_clf, X_train, X_test, y_train, y_test)
rf_accuracy, rf_precision, rf_recall = evaluate_model(rf_clf, X_train, X_test, y_train, y_test)

# 평가 결과 출력
print("Decision Tree - 정확도: {:.4f}, 정밀도: {:.4f}, 재현율: {:.4f}".format(dt_accuracy, dt_precision, dt_recall))
print("Logistic Regression - 정확도: {:.4f}, 정밀도: {:.4f}, 재현율: {:.4f}".format(lr_accuracy, lr_precision, lr_recall))
print("Random Forest - 정확도: {:.4f}, 정밀도: {:.4f}, 재현율: {:.4f}".format(rf_accuracy, rf_precision, rf_recall))

Decision Tree - 정확도: 0.7877, 정밀도: 0.6885, 재현율: 0.6885
Logistic Regression - 정확도: 0.8492, 정밀도: 0.7742, 재현율: 0.7869
Random Forest - 정확도: 0.8547, 정밀도: 0.8070, 재현율: 0.7541


In [ ]:
# - 2. dt_clf , folds=5 적용하여 KFold 교차검증 수행

from sklearn.model_selection import KFold, cross_val_score
from sklearn.tree import DecisionTreeClassifier
import numpy as np

# 결정 트리 분류기
dt_clf = DecisionTreeClassifier(random_state=11)

# KFold 교차 검증 수행
kfold = KFold(n_splits=5)
cv_accuracy = cross_val_score(dt_clf, X_titanic_df, y_titanic_df, cv=kfold)

# 평균 정확도 계산
mean_accuracy = np.mean(cv_accuracy)

print(f'결정 트리 kfold=5 교차 검증의 평균 정확도: {mean_accuracy:.4f}')


결정 트리 kfold=5 교차 검증의 평균 정확도: 0.7823


In [ ]:
# - 3. dt_clf , cv=5 적용, cross_val_score를 이용하여 교차검증 수행 
# cross_val_score 는 기본적으로 
# kfold 를 Stratified KFold 써서 잘 섞어 주고 머신러닝 돌릴 데이터가 회귀(그냥 kfold)인지, 분류(stratified kfold)인지 알아서 적합한 방식을 
# 적용해주므로 kfold를 굳이 쓸 필요 없다

from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score
import numpy as np

# 결정 트리 분류기
dt_clf = DecisionTreeClassifier(random_state=11)

# cross_val_score를 이용한 교차 검증 수행, cv=5
scores = cross_val_score(dt_clf, X_titanic_df, y_titanic_df, cv=5)

# 평균 정확도 계산
mean_accuracy = np.mean(scores)

print(f'결정 트리 cross_val_score 검증의 평균 정확도: {mean_accuracy:.4f}')


결정 트리 cross_val_score 검증의 평균 정확도: 0.7879


In [ ]:
# - 4. GridSearchCV의 최적 하이퍼 파라미터로 학습된 Estimator로 예측 및 평가 수행.
#   - parameters = {'max_depth':[2,3,5,10], 'min_samples_split':[2,3,5], 'min_samples_leaf':[1,5,8]}
#   - dt_clf, scoring='accuracy', cv=5 적용


from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score

# GridSearchCV에 사용할 하이퍼 파라미터 그리드 설정
parameters = {
    'max_depth': [2, 3, 5, 10],    
    'min_samples_leaf': [1, 5, 8],
    'min_samples_split': [2, 3, 5]
}

# DecisionTreeClassifier 객체 생성
dt_clf = DecisionTreeClassifier(random_state=11)

# GridSearchCV 객체 생성 및 학습
grid_dtree = GridSearchCV(dt_clf, param_grid=parameters, scoring='accuracy', cv=5)
grid_dtree.fit(X_train, y_train)

# 최적 하이퍼 파라미터 출력 및 최적 모델 가져오기
print('GridSearchCV 최적 하이퍼 파라미터:', grid_dtree.best_params_)
print()

best_dtree = grid_dtree.best_estimator_

# 최적 모델로 예측 및 평가 수행
pred = best_dtree.predict(X_test)
accuracy = accuracy_score(y_test, pred)

print('찾아낸 최적값으로 DecisionTreeClassifier 정확도: {0:.4f}'.format(accuracy))


GridSearchCV 최적 하이퍼 파라미터: {'max_depth': 3, 'min_samples_leaf': 5, 'min_samples_split': 2}

찾아낸 최적값으로 DecisionTreeClassifier 정확도: 0.8715


In [ ]:
#하는 김에 정확도 최적일 때 정밀도 재현율 확인해보기

from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score

# 최적 하이퍼파라미터로 결정 트리 모델 생성
dt_clf = DecisionTreeClassifier(max_depth=3, min_samples_leaf=5, min_samples_split=2, random_state=11)

# 모델 학습
dt_clf.fit(X_train, y_train)

# 테스트 데이터셋으로 예측
pred = dt_clf.predict(X_test)

# 정확도, 정밀도, 재현율 평가
accuracy = accuracy_score(y_test, pred)
precision = precision_score(y_test, pred, pos_label=1)   #pos_label=1 은 생존한걸 양성으로 지정해서 오류가 나지 않게 설정
recall = recall_score(y_test, pred, pos_label=1)


print(f'찾아낸 최적 값으로 dt 정확도: {accuracy:.4f}')
print(f'찾아낸 최적 값으로 dt 정밀도: {precision:.4f}')
print(f'찾아낸 최적 값으로 dt 재현율: {recall:.4f}')


찾아낸 최적 값으로 dt 정확도: 0.8715
찾아낸 최적 값으로 dt 정밀도: 0.8393
찾아낸 최적 값으로 dt 재현율: 0.7705
